# Quantitative Trading Strategies

In [38]:
import pandas as pd
import numpy as np
import requests 
import datetime as dt
import typing
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
import time

In [11]:
url_template = 'https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page={}'



In [57]:
trade_transactions = pd.read_csv('C:/Users/dcste/OneDrive/Economics_Research/Economics_Research/trade_transactions.csv')

In [47]:
trade_transactions

,transaction_date,disclosure_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,party,state,industry,sector,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,2/16/2023,2/16/2023,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Gary C. Peters,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/23/2023,2/14/2023,Joint,GOLD,Barrick Gold Corporation Common Stock (BC),Stock,Purchase,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries,NaN,NaN,NaN,NaN,NaN
2,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sale (Partial),"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries,NaN,NaN,NaN,NaN,NaN
3,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sale (Partial),"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries,NaN,NaN,NaN,NaN,NaN
4,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$100,001 - $250,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,8/17/2012,8/17/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8253,8/16/2012,8/16/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Pat Roberts,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8254,8/15/2012,8/15/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Rob Portman,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8255,8/2/2012,8/2/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Thomas R. Carper,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for pg in range(1,5):
    print(f'https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page={pg}')

https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page=1
https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page=2
https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page=3
https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=indices&assetType=etf&assetType=etn&assetType=reit&assetType=stock&page=4


In [51]:
trade_dataset = pd.DataFrame(data = None, columns = ['Politician','Asset_Name','Date_Traded','Filed_After','Owner','Trade_Type','Notional_Value','Price'])

In [52]:
def filter_tradesize(trade_size):
    value = trade_size.strip()
    if value == '<1K' or value == '< 1K':
        value = 500
    elif value == '1K\u201315K':
        value = np.mean([1e3,15e3])
    elif value == '15K\u201350K':
        value = np.mean([15e3,50e3])
    elif value == '50K\u2013100K':
        value = np.mean([50e3,100e3])
    elif value == '100K\u2013250K':
        value = np.mean([100e3,250e3])
    elif value == '250K\u2013500K':
        value = np.mean([250e3,500e3])
    elif value == '1M\u20135M':
        value = np.mean([1e6,5e6])
    elif value == '5M\u201325M':
        value = np.mean([5e6,25e6])
    elif value == '25M\u201350M':
        value = np.mean([25e6,50e6])
    
    return value


In [53]:
def filter_dates(dates: str):
    DATE = dates
    d_format = "%Y %d %b"
    output_format = "%m/%d/%Y"
    date_ob = dt.datetime.strptime(DATE,d_format)
    final_ob = date_ob.strftime(output_format)
    return final_ob
    

In [243]:
filter_dates('2023 29 Jan')

'01/29/2023'

In [203]:
filter_tradesize('25M-50M')

37500000.0

In [54]:

row_index = 0
page_num = 1
for pg in range(1,10):
    url = "https://www.capitoltrades.com/trades?mcap=3&mcap=2&mcap=1&assetType=abs&assetType=corporate-bond&assetType=etf&assetType=stock&assetType=reit&assetType=indices&assetType=etn&page={pg}"
   
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find('table')
    rows = table.find_all('tr')


    if len(rows) == 1:
        break

    for row in rows[1:]:
        cols = row.find_all('td')

        if len(cols) > 0:
        # Extract the data from the columns
            #temp_df = pd.DataFrame(data = None,columns = ['Politician','Asset_Name','Date_Traded','Filed_After','Owner','Trade_Type','Notional_Value','Price'])
            Politician = cols[0].text.strip()
            Asset_Name = cols[1].text.strip()
            #published = cols[2].text.strip()
            Date_Traded = filter_dates(cols[3].text.strip())
            try:
                Filed_After = float(cols[4].text.strip()[-2:])
            except ValueError:
                Filed_After = np.nan
            Owner= cols[5].text.strip()
            Trade_Type = cols[6].text.strip()
            Notional_Value = filter_tradesize(cols[7].text.strip())

            try:
                Price = float(cols[8].text.strip())
            except ValueError:
                Price = np.nan
            #print(politician,asset_name,date_traded, filed_after, owner, trade_type, trade_size, price)
                       # Append the extracted data to the DataFrame
            trade_dataset.loc[row_index] = [Politician,Asset_Name,Date_Traded,Filed_After,Owner,Trade_Type,Notional_Value,Price]

            row_index += 1
            
            
         



    
            



In [56]:
trade_dataset.drop_duplicates('Price')

,Politician,Asset_Name,Date_Traded,Filed_After,Owner,Trade_Type,Notional_Value,Price
0,Jeff JacksonDemocratHouseNC,ANKR $ANKR,01/29/2023,19.0,Undisclosed,sell,8000.0,0.028
1,Jeff JacksonDemocratHouseNC,CARDANO$ADA,01/29/2023,19.0,Undisclosed,sell,8000.0,0.396
2,Neal DunnRepublicanHouseFL,KeyCorpKEY:US,01/05/2023,43.0,Undisclosed,sell,8000.0,22.040
3,Neal DunnRepublicanHouseFL,Regions Financial CorpRF:US,01/05/2023,43.0,Undisclosed,sell,8000.0,NaN
4,Neal DunnRepublicanHouseFL,The Southern CoSO:US,01/05/2023,43.0,Undisclosed,sell,32500.0,70.380
6,Josh GottheimerDemocratHouseNJ,AbbVie IncABBV:US,01/27/2023,21.0,Joint,sell,8000.0,146.280
7,Josh GottheimerDemocratHouseNJ,AbbVie IncABBV:US,01/20/2023,28.0,Joint,sell,8000.0,149.590
8,Josh GottheimerDemocratHouseNJ,Adobe IncADBE:US,01/31/2023,17.0,Joint,buy,8000.0,370.340
9,Josh GottheimerDemocratHouseNJ,Align Technology IncALGN:US,01/27/2023,21.0,Joint,buy,8000.0,269.200
10,Josh GottheimerDemocratHouseNJ,Allegro Microsystems IncALGM:US,01/23/2023,25.0,Joint,buy,8000.0,33.440
